In [1]:
# =========================================
# Predicción de temperatura futura con LSTM
# =========================================

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import joblib

# 1. Cargar datos correctamente
df = pd.read_csv("dataset_ml.csv", sep=";", decimal=".", encoding="utf-8")

# Quitar espacios extra en nombres de columnas
df.columns = df.columns.str.strip()

print("✅ Columnas detectadas:", df.columns)

# 2. Procesar columna de tiempo si existe
if 'momento' in df.columns:
    df['momento'] = pd.to_datetime(df['momento'], errors='coerce')
    cols = df.columns.drop('momento')
else:
    cols = df.columns

# 3. Convertir a numérico y limpiar valores inválidos
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')
df = df.replace([np.inf, -np.inf], np.nan).dropna()

# Verificar cantidad de filas
print(f"📊 Total de filas luego de limpiar: {len(df)}")

# 4. Escalado de todas las variables de entrada
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df[cols])

# Guardar scaler para uso futuro
joblib.dump(scaler, "scaler_clima.pkl")

# 5. Crear secuencias de tiempo
def crear_secuencias(datos, n_pasos, columna_objetivo):
    X, y = [], []
    for i in range(n_pasos, len(datos)):
        X.append(datos[i - n_pasos:i])
        y.append(datos[i, columna_objetivo])
    return np.array(X), np.array(y)

# Usaremos 24 pasos (por ejemplo, 24 horas si los datos son horarios)
n_pasos = 24
# Buscar el índice de la columna 'ts' (temperatura) en el escalado
columna_objetivo = list(cols).index('ts')

X, y = crear_secuencias(scaled_data, n_pasos, columna_objetivo)

print(f"✅ Secuencias creadas: X={X.shape}, y={y.shape}")

# 6. Dividir en entrenamiento y test
porc_entrenamiento = 0.8
n_train = int(len(X) * porc_entrenamiento)

X_train, X_test = X[:n_train], X[n_train:]
y_train, y_test = y[:n_train], y[n_train:]

# 7. Construcción del modelo LSTM
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape=(n_pasos, X.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(1))  # salida: temperatura futura

model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

# 8. Entrenamiento
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

# 9. Evaluación del modelo
loss, mae = model.evaluate(X_test, y_test)
print(f"📉 MAE en test: {mae:.2f} °C")

# 10. Predicción futura usando la última ventana
ultima_secuencia = X[-1].reshape(1, n_pasos, X.shape[2])
prediccion_escalada = model.predict(ultima_secuencia)[0][0]

# 11. Invertir el escalado para obtener la temperatura real
dummy = np.zeros((1, scaled_data.shape[1]))
dummy[0, columna_objetivo] = prediccion_escalada
prediccion_real = scaler.inverse_transform(dummy)[0, columna_objetivo]

print(f"🌡️ Temperatura predicha a futuro: {prediccion_real:.2f} °C")

# 12. Guardar modelo y scaler
model.save("modelo_lstm_clima.h5")
print("✅ Modelo guardado como 'modelo_lstm_clima.h5'")
print("✅ Scaler guardado como 'scaler_clima.pkl'")



✅ Columnas detectadas: Index(['momento', 'ts', 'td', 'tMin12Horas', 'tMax12Horas', 'tMin24Horas',
       'hr', 'p0', 'qfe1', 'qfe2', 'qff', 'qnh', 'tPromedio24h', 'deltaTemp1h',
       'deltaPresion1h', 'humedadRelativaCambio'],
      dtype='object')
📊 Total de filas luego de limpiar: 96566
✅ Secuencias creadas: X=(96542, 24, 15), y=(96542,)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 24s 8ms/step - loss: 0.0238 - mae: 0.0956 - val_loss: 0.0025 - val_mae: 0.0431
Epoch 2/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 19s 8ms/step - loss: 0.0065 - mae: 0.0594 - val_loss: 0.0012 - val_mae: 0.0272
Epoch 3/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 21s 8ms/step - loss: 0.0062 - mae: 0.0575 - val_loss: 5.9358e-04 - val_mae: 0.0195
Epoch 4/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 19s 8ms/step - loss: 0.0059 - mae: 0.0560 - val_loss: 0.0017 - val_mae: 0.0325
Epoch 5/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 21s 9ms/step - loss: 0.0057 - mae: 0.0548 - val_loss: 9.9663e-04 - val_mae: 0.0248
Epoch 6/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 19s 8ms/step - loss: 0.0055 - mae: 0.0540 - val_loss: 5.3553e-04 - val_mae: 0.0182
Epoch 7/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 21s 9ms/step - loss: 0.0054 - mae: 0.0531 - val_loss: 0.0012 - val_mae: 0.0295
Epoch 8/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step - loss: 0.0055 - mae: 0.0534 - val_loss: 8.5169e-04 - val_mae: 0.0223
Epoch 9/

🌡️ Temperatura predicha a futuro: 14.09 °C
✅ Modelo guardado como 'modelo_lstm_clima.h5'
✅ Scaler guardado como 'scaler_clima.pkl'


In [2]:
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 716.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 92.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 135.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 122.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.3 MB/s eta 0:00:00


In [3]:
# =========================================
# Predicción de temperatura futura con GRU
# =========================================

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

model_gru = Sequential()
model_gru.add(GRU(128, return_sequences=True, input_shape=(n_pasos, X.shape[2])))
model_gru.add(Dropout(0.2))
model_gru.add(GRU(64))
model_gru.add(Dropout(0.2))
model_gru.add(Dense(1))

model_gru.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history_gru = model_gru.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

loss_gru, mae_gru = model_gru.evaluate(X_test, y_test)
print(f"📉 MAE GRU: {mae_gru:.2f} °C")

model_gru.save("modelo_gru_clima.h5")
print("✅ Modelo GRU guardado como 'modelo_gru_clima.h5'")


Epoch 1/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step - loss: 0.0242 - mae: 0.1005 - val_loss: 0.0028 - val_mae: 0.0449
Epoch 2/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 17s 7ms/step - loss: 0.0066 - mae: 0.0595 - val_loss: 0.0010 - val_mae: 0.0247
Epoch 3/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - loss: 0.0058 - mae: 0.0557 - val_loss: 9.8220e-04 - val_mae: 0.0251
Epoch 4/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 19s 8ms/step - loss: 0.0056 - mae: 0.0548 - val_loss: 9.9329e-04 - val_mae: 0.0256
Epoch 5/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - loss: 0.0054 - mae: 0.0533 - val_loss: 9.4119e-04 - val_mae: 0.0241
Epoch 6/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - loss: 0.0052 - mae: 0.0525 - val_loss: 6.2185e-04 - val_mae: 0.0201
Epoch 7/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 17s 7ms/step - loss: 0.0052 - mae: 0.0521 - val_loss: 9.3193e-04 - val_mae: 0.0241
Epoch 8/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - loss: 0.0053 - mae: 0.0527 - val_loss: 5.6385e-04 - val_mae: 0.0184


📉 MAE GRU: 0.01 °C
✅ Modelo GRU guardado como 'modelo_gru_clima.h5'


In [4]:
# =========================================
# Modelo híbrido CNN + LSTM
# =========================================

from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout

model_cnn_lstm = Sequential()
model_cnn_lstm.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_pasos, X.shape[2])))
model_cnn_lstm.add(MaxPooling1D(pool_size=2))
model_cnn_lstm.add(LSTM(64))
model_cnn_lstm.add(Dropout(0.3))
model_cnn_lstm.add(Dense(1))

model_cnn_lstm.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history_cnnlstm = model_cnn_lstm.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

loss_cnnlstm, mae_cnnlstm = model_cnn_lstm.evaluate(X_test, y_test)
print(f"📉 MAE CNN+LSTM: {mae_cnnlstm:.2f} °C")

model_cnn_lstm.save("modelo_cnn_lstm_clima.h5")
print("✅ Modelo CNN+LSTM guardado como 'modelo_cnn_lstm_clima.h5'")


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 18s 6ms/step - loss: 0.0360 - mae: 0.1170 - val_loss: 0.0017 - val_mae: 0.0326
Epoch 2/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 15s 6ms/step - loss: 0.0095 - mae: 0.0711 - val_loss: 0.0027 - val_mae: 0.0414
Epoch 3/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 15s 6ms/step - loss: 0.0092 - mae: 0.0693 - val_loss: 0.0013 - val_mae: 0.0278
Epoch 4/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - loss: 0.0089 - mae: 0.0677 - val_loss: 0.0012 - val_mae: 0.0266
Epoch 5/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 15s 6ms/step - loss: 0.0083 - mae: 0.0658 - val_loss: 0.0019 - val_mae: 0.0329
Epoch 6/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 15s 6ms/step - loss: 0.0083 - mae: 0.0657 - val_loss: 0.0014 - val_mae: 0.0314
Epoch 7/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - loss: 0.0083 - mae: 0.0651 - val_loss: 8.9853e-04 - val_mae: 0.0229
Epoch 8/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - loss: 0.0080 - mae: 0.0642 - val_loss: 7.8389e-04 - val_mae: 0.0215
Epoch 9/100
2414

📉 MAE CNN+LSTM: 0.02 °C
✅ Modelo CNN+LSTM guardado como 'modelo_cnn_lstm_clima.h5'


In [6]:
# =========================================
# Modelo basado en Transformer simple
# =========================================

from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, Flatten
import tensorflow as tf
inputs = tf.keras.Input(shape=(n_pasos, X.shape[2]))
x = MultiHeadAttention(num_heads=4, key_dim=16)(inputs, inputs)
x = LayerNormalization(epsilon=1e-6)(x)
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.3)(x)
outputs = Dense(1)(x)

model_transformer = tf.keras.Model(inputs, outputs)
model_transformer.compile(optimizer=Adam(0.001), loss='mse', metrics=['mae'])

history_trans = model_transformer.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

loss_trans, mae_trans = model_transformer.evaluate(X_test, y_test)
print(f"📉 MAE Transformer: {mae_trans:.2f} °C")

model_transformer.save("modelo_transformer_clima.h5")
print("✅ Modelo Transformer guardado como 'modelo_transformer_clima.h5'")


Epoch 1/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 16s 5ms/step - loss: 0.1362 - mae: 0.2151 - val_loss: 0.0280 - val_mae: 0.1340
Epoch 2/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - loss: 0.0300 - mae: 0.1266 - val_loss: 0.0097 - val_mae: 0.0763
Epoch 3/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - loss: 0.0273 - mae: 0.1207 - val_loss: 0.0057 - val_mae: 0.0601
Epoch 4/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - loss: 0.0262 - mae: 0.1183 - val_loss: 0.0035 - val_mae: 0.0486
Epoch 5/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - loss: 0.0255 - mae: 0.1160 - val_loss: 0.0038 - val_mae: 0.0494
Epoch 6/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - loss: 0.0248 - mae: 0.1139 - val_loss: 0.0076 - val_mae: 0.0694
Epoch 7/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - loss: 0.0245 - mae: 0.1131 - val_loss: 0.0067 - val_mae: 0.0655
Epoch 8/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - loss: 0.0239 - mae: 0.1108 - val_loss: 0.0049 - val_mae: 0.0591
Epoch 9/100
2414/2414 ━━━━━━━━━

📉 MAE Transformer: 0.02 °C
✅ Modelo Transformer guardado como 'modelo_transformer_clima.h5'
